
# Using Le_net class to train and save the model 

In [1]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Flatten, Dense
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import os

class LeNet:
    def __init__(self, batch_size=32, epochs=20):
        self.batch_size = batch_size
        self.epochs = epochs
        self.model = None
        self._create_lenet()
        self._compile()

    def _create_lenet(self):
    
        self.model = Sequential([
            Conv2D(filters=6, kernel_size=(5, 5), activation='sigmoid',
                   input_shape=(28, 28, 1), padding='same'),
            AveragePooling2D(pool_size=(2, 2), strides=2),
            
            Conv2D(filters=16, kernel_size=(5, 5), activation='sigmoid', padding='same'),
            AveragePooling2D(pool_size=(2, 2), strides=2),
            
            Flatten(),
            Dense(120, activation='sigmoid'),
            Dense(84, activation='sigmoid'),
            Dense(10, activation='softmax')
        ])

    def _compile(self):
        """
        Compile the LeNet model with an Adam optimizer and categorical crossentropy loss.
        """
        if self.model is None:
            raise ValueError('Error: Model is not created yet.')
        
        self.model.compile(optimizer='Adam',
                           loss='categorical_crossentropy',
                           metrics=['accuracy'])

    def _preprocess(self):
        """
        Preprocess the MNIST dataset: normalize, reshape, and one-hot encode the labels.
        """
        (x_train, y_train), (x_test, y_test) = mnist.load_data()
        
        # Normalize the pixel values to the range [0, 1]
        x_train = x_train / 255.0
        x_test = x_test / 255.0
        
        # Add channel dimension
        self.x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
        self.x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
        
        # One-hot encode the labels
        self.y_train = to_categorical(y_train, 10)
        self.y_test = to_categorical(y_test, 10)

    def train(self):
        """
        Train the model using the preprocessed MNIST dataset.
        """
        self._preprocess()
        self.model.fit(self.x_train, self.y_train, 
                       batch_size=self.batch_size, 
                       epochs=self.epochs,
                       validation_data=(self.x_test, self.y_test))

    def save(self, model_path_name):
        """
        Save the trained model to the specified file path.
        """
        if not model_path_name.endswith(".keras"):
            model_path_name += ".keras"
        self.model.save(model_path_name)
        print(f"Model saved as {model_path_name}")

    def load(self, model_path_name):
        """
        Load a saved model from the specified file path.
        """
        if not os.path.exists(model_path_name):
            raise FileNotFoundError(f"No such file: {model_path_name}")
        self.model = load_model(model_path_name)
        print(f"Model loaded from {model_path_name}")

    def predict(self, images):
        """
        Predict the class probabilities for a list of images.
        """
        if self.model is None:
            raise ValueError("Model is not loaded or created yet.")
        
        # Preprocess images: normalize and add channel dimension
        images = [img / 255.0 for img in images]
        images = [img.reshape(1, 28, 28, 1) for img in images]
        
        # Run predictions and return results
        predictions = [self.model.predict(img, verbose=0) for img in images]
        return [pred.argmax() for pred in predictions]

In [2]:
lenet = LeNet(batch_size=64, epochs=10)

c:\Users\rishi\anaconda3\envs\ece5831-2024\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [3]:
lenet.train()

Epoch 1/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.3668 - loss: 1.7505 - val_accuracy: 0.9073 - val_loss: 0.3145
Epoch 2/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9161 - loss: 0.2823 - val_accuracy: 0.9428 - val_loss: 0.1941
Epoch 3/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9441 - loss: 0.1806 - val_accuracy: 0.9616 - val_loss: 0.1290
Epoch 4/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9592 - loss: 0.1341 - val_accuracy: 0.9665 - val_loss: 0.1029
Epoch 5/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9680 - loss: 0.1054 - val_accuracy: 0.9705 - val_loss: 0.0964
Epoch 6/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9736 - loss: 0.0863 - val_accuracy: 0.9789 - val_loss: 0.0693
Epoch 7/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9772 - loss: 0.0757 - val_accuracy: 0.9785 - val_loss: 0.0672
Epoch 8/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9794 - loss: 0.0638 - val_accuracy: 0.

In [13]:
lenet.save("RDewalkar")

Model saved as RDewalkar.keras


In [14]:
lenet.load("RDewalkar.keras")

Model loaded from RDewalkar.keras


In [6]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [7]:
x_test.shape

(10000, 28, 28)

In [8]:
type(x_test)

numpy.ndarray

In [9]:
x_test[0:10].shape

(10, 28, 28)

In [10]:
prediction = lenet.predict(x_test[0:10])

In [11]:
prediction

[np.int64(7),
 np.int64(2),
 np.int64(1),
 np.int64(0),
 np.int64(4),
 np.int64(1),
 np.int64(4),
 np.int64(9),
 np.int64(5),
 np.int64(9)]

## Testing our Hand Written Images

In [12]:
import numpy as np
import os
import cv2

import os
import cv2

def process_images_from_folder(folder_path):
    """
    Load and process all images from a folder, ensuring each image is (28, 28) (grayscale).

    Args:
        folder_path (str): Path to the folder containing images.

    Returns:
        list: A list of processed images, each with shape (28, 28).
    """
    processed_images = []
    
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        
        # Ensure it's a valid image file
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image = cv2.imread(file_path)  # Load image
            
            if image is not None:  # Check if image is loaded successfully
                # Convert to grayscale (if it's not already)
                grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                
                # Resize to 28x28
                resized_image = cv2.resize(grayscale_image, (28, 28))
                
                # Append to the list
                processed_images.append(resized_image)
    
    return processed_images

In [ ]:
folder_path = "D://Masters//Pattern Recognition and Neural Networks//0_images" 
test_images = process_images_from_folder(folder_path)

# Predict with LeNet
predictions = lenet.predict(test_images)
print(predictions)